In [1]:
import copy
import re
from iteration_utilities import deepflatten

In [2]:
def is_number(s):
    try:
        float(s) 
    except ValueError:
        try:
            complex(s) 
        except ValueError:
            return False
    return True

In [3]:
class Frame:
    def __init__(self,arg=None):
        if arg is None:
            self._global=dict()
            self._atomic=dict(dict())
        if isinstance(arg,Frame):
            self._global=copy.deepcopy(arg._global)
            self._atomic=copy.deepcopy(arg._atomic)
    
    def __getitem__(self,key1):
        index,key=key1
        if index == -1:
            return self._global[key]
        else: 
            return self._atomic[index][key]
        
    def __setitem__(self,key1,val):
        index,key=key1
        if index == -1:
            self._global.update({key:val})
        else: 
            if index in self._atomic:
                self._atomic[index].update({key:val})
            else:
                empt=dict()
                self._atomic.update({index:empt})
                self._atomic[index].update({key:val})
        

In [4]:
# Code does not check if atom index is repeated 
# or missing data entries. 
def read_reax_traj(filename,
                   global_names=['Step','N','Lx_min','Lx_max','Ly_min','Ly_max','Lz_min','Lz_max'],
                   atom_names=['type','x','y','z','vx','vy','vz'],
                   type_list=['','O','H']):
    comment_pattern=re.compile(r"\n")
    data_pattern=re.compile(r"\s")
    inHeader=True
    frame=Frame()
    traj=[]
    with open(filename, 'r') as file:
        line=file.readline()
        while line:
            if inHeader:
                line_items=comment_pattern.split(line)
            else:
                line_items=data_pattern.split(line)
                
            
            if not inHeader:
                if not is_number(line_items[0]):
                    if count != N_actual:
                        raise ValueError(f"{count} lines, instead of {N_actual}")
                        
                    inHeader=True
                    line_items=comment_pattern.split(line)
                else:
                    ind=int(line_items[0])
                    frame[ind,atom_names[0]]=type_list[int(line_items[1])]
                    frame[ind,atom_names[1]]=float(line_items[2])
                    frame[ind,atom_names[2]]=float(line_items[3])
                    frame[ind,atom_names[3]]=float(line_items[4])
                    frame[ind,atom_names[4]]=float(line_items[5])
                    frame[ind,atom_names[5]]=float(line_items[6])
                    frame[ind,atom_names[6]]=float(line_items[7])
                    count+=1

            if line_items[0] == 'ITEM: TIMESTEP':
                traj.append(Frame(frame))
                frame=Frame()
                line=file.readline()
                line_items=data_pattern.split(line)
                frame[-1,global_names[0]]=int(line_items[0])
            
            if line_items[0] == 'ITEM: NUMBER OF ATOMS':
                line=file.readline()
                line_items=data_pattern.split(line)
                frame[-1,global_names[1]]=int(line_items[0])
                N_actual=int(line_items[0])
                
            if line_items[0] == 'ITEM: BOX BOUNDS pp pp pp':
                line=file.readline()
                line_items=data_pattern.split(line)
                frame[-1,global_names[2]]=float(line_items[0])
                frame[-1,global_names[3]]=float(line_items[1])
                line=file.readline()
                line_items=data_pattern.split(line)
                frame[-1,global_names[4]]=float(line_items[0])
                frame[-1,global_names[5]]=float(line_items[1])
                line=file.readline()
                line_items=data_pattern.split(line)
                frame[-1,global_names[6]]=float(line_items[0])
                frame[-1,global_names[7]]=float(line_items[1])
                
            if line_items[0] == 'ITEM: ATOMS id type x y z vx vy vz':
                inHeader=False
                count=0
            
            line=file.readline()
            
        traj.append(Frame(frame))
        traj.pop(0)
    return traj

In [5]:
def read_reax_log_traj(filename):
    
    def process_line(line):
        data_pattern=re.compile(r"\s+")
        line=data_pattern.sub(" ",line).strip()
        return data_pattern.split(line)
        
    inHeader=True
    key_items=[]
    key_items_len=0
    frame=Frame()
    traj=[]
    with open(filename, 'r') as file:
        line=file.readline()
        line_items=process_line(line)
        
        while line_items[0] != 'Step':
            line=file.readline()
            line_items=process_line(line)
            
        key_items=list(line_items)
        key_items_len=len(key_items)
        
        line=file.readline()
        line_items=process_line(line)
        line_items_len=len(line_items)
        
        while line_items_len == key_items_len:
            traj.append(Frame(frame))
            frame=Frame()
            
            frame[-1,key_items[0]]=int(line_items[0])
            
            for ind in range(1,key_items_len):
                frame[-1,key_items[ind]]=float(line_items[ind])
                
            line=file.readline()
            line_items=process_line(line)
            line_items_len=len(line_items)
            
            
        traj.append(Frame(frame))
        traj.pop(0)
    return traj

In [6]:
def projections(traj,time_keys,index_keys,val_keys,depth=1):
    l=list()
    for frame in traj:
        if frame[-1,'Step'] in time_keys:
            index_list=list()
            for index in index_keys:
                val_list=list()
                for val in val_keys:
                    val_list.append(frame[index,val])
                index_list.append(list(val_list))
            l.append(list(index_list))
    if depth == 0:
        return l;
    else:
        return list(deepflatten(l,depth=depth))

In [7]:
trj=read_reax_traj("test_traj.txt")
print(trj)

[<__main__.Frame object at 0x105fef040>, <__main__.Frame object at 0x105f44b50>]


In [8]:
trj_log=read_reax_log_traj("test_log.txt")
print(trj_log)

[<__main__.Frame object at 0x105f449d0>, <__main__.Frame object at 0x105f44580>, <__main__.Frame object at 0x105f44370>, <__main__.Frame object at 0x105f44b20>, <__main__.Frame object at 0x105f44e50>, <__main__.Frame object at 0x105f449a0>, <__main__.Frame object at 0x105f44250>, <__main__.Frame object at 0x107335e20>]


In [9]:
l=projections(trj,[0,100],[4915,4916],['x','y','z'],depth=0)
print(l)

[[[4.69, 4.38, 3.83], [5.69, 4.47, 3.85]], [[9.69, 9.38, 3.83], [8.69, 8.47, 3.85]]]


In [10]:
types=projections(trj,[0,100],[4915,4916,4917,9905],['type'],depth=2)
print(types)

['O', 'H', 'H', 'H', 'O', 'H', 'H', 'H']


In [11]:
number=projections(trj,[0,100],[-1],['N'],depth=-1)
print(number)

[4, 4]


In [12]:
data=projections(trj_log,[100,200,300],[-1],
                  ['Temp', 'PotEng', 'TotEng', 'Press', 'Volume', 'Density', 'Enthalpy'],
                   depth=1)
print(data)

[[177.26875, -4308271.0, -4282366.0, -6290.0336, 487140.35, 1.0035394, -4327053.2], [192.50222, -4309154.8, -4281023.6, 5959.734, 483875.68, 1.0103102, -4238966.8], [200.85192, -4308692.4, -4279341.0, -2143.447, 482460.56, 1.0132736, -4294422.7]]
